<a href="https://colab.research.google.com/github/FernandoBRdgz/diplomado_ds_mod4/blob/main/4.6%20Modelaci%C3%B3n%20supervisada%20y%20no%20supervisada%20de%20Big%20Data/4.6.5%20Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Clustering

Los ingenieros de una empresa de tecnología que ha sido hackeada obtuvieron afortunadamente datos valiosos sobre los ataques, incluida información como la hora de la sesión, las ubicaciones, la velocidad de escritura de palabras por minuto, datos de cada sesión que los hackers utilizaron para conectarse a sus servidores.

---

### Conjunto de Datos

* Session_Connection_Time: Duración de la sesión en minutos
* Bytes Transferred: Cantidad de MB transferidos durante la sesión
* Kali_Trace_Used: Indica si el hacker estaba usando Kali Linux
* Servers_Corrupted: Número de servidores corrompidos durante el ataque
* Pages_Corrupted: Número de páginas a las que se ha accedido ilegalmente
* Location: Ubicación de la que provino el ataque (probablemente inútil por el uso de VPN)
* WPM_Typing_Speed: Velocidad de escritura estimada en función de los registros de sesión. (WPM - Words Per Minute)

---

La firma de tecnología estima que fueron 3 los posibles hackers que perpetraron el ataque. Están seguros de los dos primeros, pero no están muy seguros de si un tercero estuvo involucrado o no. Se busca averiguar si el tercer sospechoso tuvo algo que ver con los ataques o si fueron solo dos.

Como último dato clave, uno de los ingenieros sabe que los hackers intercambian ataques, esto significa que cada uno debería tener aproximadamente la misma cantidad de ataques. Por ejemplo, si hubo 100 ataques en total, entonces en el escenario con 2 hackers, cada uno debería tener alrededor de 50 ataques, en un escenario con tres hackers, cada uno tendría alrededor de 33 ataques.

In [ ]:
generation = "mod4gen<x>"

In [ ]:
data = spark.read.csv(f"/mnt/{generation}/input/hack_data.csv", inferSchema=True, header=True)

In [ ]:
data.count(), len(data.columns)

In [ ]:
data.display()

In [ ]:
data.describe().display()

### EDA

In [ ]:
data.select("Session_Connection_Time").display()

In [ ]:
data.select("Bytes Transferred").display()

In [ ]:
data.select("Kali_Trace_Used").display()

In [ ]:
data.select("Servers_Corrupted").display()

In [ ]:
data.select("Pages_Corrupted").orderBy("Pages_Corrupted").display()

### Preprocesamiento

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
cols = ['Session_Connection_Time', 'Bytes Transferred', 'Kali_Trace_Used', 'Servers_Corrupted', 'Pages_Corrupted','WPM_Typing_Speed']

In [ ]:
vec_assembler = VectorAssembler(inputCols = cols, outputCol="features")

In [ ]:
final_data = vec_assembler.transform(data)

In [ ]:
final_data.display()

In [ ]:
from pyspark.ml.feature import StandardScaler

In [ ]:
help(StandardScaler)

In [ ]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)

In [ ]:
scalerModel = scaler.fit(final_data)

In [ ]:
cluster_final_data = scalerModel.transform(final_data)

In [ ]:
cluster_final_data.display()

### Modelación

In [ ]:
from pyspark.ml.clustering import KMeans

In [ ]:
kmeans3 = KMeans(featuresCol="scaledFeatures", k=3)
kmeans2 = KMeans(featuresCol="scaledFeatures", k=2)

In [ ]:
model_k3 = kmeans3.fit(cluster_final_data)
model_k2 = kmeans2.fit(cluster_final_data)

In [ ]:
from pyspark.ml.evaluation import ClusteringEvaluator

[ClusteringEvaluator](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.evaluation.ClusteringEvaluator.html)

In [ ]:
silhouette_score=[]
evaluator = ClusteringEvaluator(predictionCol="prediction", featuresCol="scaledFeatures", metricName="silhouette", distanceMeasure="squaredEuclidean")

for k in range(2,9):
    kmeans = KMeans(featuresCol="scaledFeatures", k=k)
    kmeans_fit = kmeans.fit(cluster_final_data)
    output = kmeans_fit.transform(cluster_final_data)
    score = evaluator.evaluate(output)
    silhouette_score.append(score)
    print(f"Silhouette Score k={k}:", score)

In [ ]:
scores = spark.createDataFrame(list(zip(range(2,9), silhouette_score)), ["k", "score"])
scores.display()

In [ ]:
scores.display()

No se puede tomar una decisión definitiva de lo anterior. Pero recordemos que el ingeniero mencionó que los ataques deberían contarse por igual entre los hackers. Esto se puede responder con los métodos de transformación y predicción.

In [ ]:
model_k3.transform(cluster_final_data).groupBy("prediction").count().display()

In [ ]:
model_k2.transform(cluster_final_data).groupBy("prediction").count().display()

En efecto, fueron 2 hackers, de hecho, nuestro algoritmo de clustering creó dos grupos del mismo tamaño con K = 2, ¡no hay forma de que sea una coincidencia!